In [ ]:
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pytesseract as tes
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
cap = cv2.VideoCapture("Videos/Full_font1.avi")

In [ ]:
## Saving
# fps =int(cap.get(cv2.CAP_PROP_FPS))
# diap = tqdm(iterable=range(0,fps*40,fps))
# for i in diap:
#     cap.set(cv2.CAP_PROP_POS_FRAMES, i)
#     ret, frame = cap.read()
#     if ret:
#         cv2.imwrite(f"Images/frame{int(i/fps)}.jpg",frame)

In [77]:
def recognize(image,thresh):
    frame_2color = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh, frame_bw = cv2.threshold(frame_2color, thresh, 255, cv2.THRESH_BINARY)
    return tes.image_to_string(frame_bw),frame_bw

In [78]:
# Check
THRESH = 147

pdf_file = PdfPages('Full_font1.pdf')

fps = int(cap.get(cv2.CAP_PROP_FPS))
diap = tqdm(iterable=range(0, fps * 40, fps))

for i in diap:
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ready, frame = cap.read()
    text, frame_treated = recognize(frame, thresh=THRESH)

    if ready:
        fig, (ax_row, ax_treated) = plt.subplots(ncols=2)

        ax_row.imshow(frame)
        ax_treated.imshow(frame_treated)
        ax_row.text(0, 0, text, fontsize=15)
        
        fig.savefig(pdf_file, format='pdf')
        plt.close()
pdf_file.close()

100%|██████████| 40/40 [00:11<00:00,  3.41it/s]
